Pandas Applying Functions


Load data


In [ ]:
# Importing Libraries
import pandas as pd
from datasets import load_dataset
import matplotlib.pyplot as plt

# Loading Data
dataset = load_dataset('lukebarousse/data_jobs')
df = dataset['train'].to_pandas()

# Data Cleanup
df['job_posted_date'] = pd.to_datetime(df['job_posted_date'])

Apply


In [2]:
help(df.apply)

Help on method apply in module pandas.core.frame:

apply(
    func: 'AggFuncType',
    axis: 'Axis' = 0,
    raw: 'bool' = False,
    result_type: "Literal['expand', 'reduce', 'broadcast'] | None" = None,
    args=(),
    by_row: "Literal[False, 'compat']" = 'compat',
    engine: "Literal['python', 'numba']" = 'python',
    engine_kwargs: 'dict[str, bool] | None' = None,
    **kwargs
) method of pandas.core.frame.DataFrame instance
    Apply a function along an axis of the DataFrame.

    Objects passed to the function are Series objects whose index is
    either the DataFrame's index (``axis=0``) or the DataFrame's columns
    (``axis=1``). By default (``result_type=None``), the final return type
    is inferred from the return type of the applied function. Otherwise,
    it depends on the `result_type` argument.

    Parameters
    ----------
    func : function
        Function to apply to each column or row.
    axis : {0 or 'index', 1 or 'columns'}, default 0
        Axis along wh

Notes

- `apply()`: Apply functions to columns or rows.


Example 1

Calculate projected salaries next year, using an assumed rate of 3.0% for all roles.


In [3]:
def inflation(salary):
    return salary * 1.03  # Assuming a 3% annual inflation rate


df['salary_year_inflated'] = df['salary_year_avg'].apply(inflation)

df[pd.notna(df['salary_year_avg'])][[
    'salary_year_avg', 'salary_year_inflated']]

,salary_year_avg,salary_year_inflated
28,109500.0,112785.00
77,140000.0,144200.00
92,120000.0,123600.00
100,228222.0,235068.66
109,89000.0,91670.00
...,...,...
785624,139216.0,143392.48
785641,150000.0,154500.00
785648,221875.0,228531.25
785682,157500.0,162225.00


We can actually simplify this with a lambda function.


In [4]:
df['salary_year_inflated'] = df['salary_year_avg'].apply(
    lambda salary: salary * 1.03)

df[pd.notna(df['salary_year_avg'])][[
    'salary_year_avg', 'salary_year_inflated']]

,salary_year_avg,salary_year_inflated
28,109500.0,112785.00
77,140000.0,144200.00
92,120000.0,123600.00
100,228222.0,235068.66
109,89000.0,91670.00
...,...,...
785624,139216.0,143392.48
785641,150000.0,154500.00
785648,221875.0,228531.25
785682,157500.0,162225.00


Now technically this could have been done like this...


In [5]:
df['salary_year_avg'] = df['salary_year_avg'] * 1.03

df[pd.notna(df['salary_year_avg'])][[
    'salary_year_avg', 'salary_year_inflated']]

,salary_year_avg,salary_year_inflated
28,112785.00,112785.00
77,144200.00,144200.00
92,123600.00,123600.00
100,235068.66,235068.66
109,91670.00,91670.00
...,...,...
785624,143392.48,143392.48
785641,154500.00,154500.00
785648,228531.25,228531.25
785682,162225.00,162225.00


Example 2

Calculate projected salaries next year, but:

- For senior roles (e.g., Senior Data Analysts), assume the rate is 5%
- For all other roles, assume rate is 3%


In [6]:
def projected_salary(row):
    # if pd.isna(row['salary_year_avg']):
    #     return None
    if 'Senior' in row['job_title_short']:
        return row['salary_year_avg'] * 1.05
    else:
        return row['salary_year_avg'] * 1.03


df['salary_year_inflated'] = df.apply(projected_salary, axis=1)

df[pd.notna(df['salary_year_avg'])][['job_title_short',
                                     'salary_year_avg', 'salary_year_inflated']]

,job_title_short,salary_year_avg,salary_year_inflated
28,Data Scientist,112785.00,116168.5500
77,Data Engineer,144200.00,148526.0000
92,Data Engineer,123600.00,127308.0000
100,Data Scientist,235068.66,242120.7198
109,Data Analyst,91670.00,94420.1000
...,...,...,...
785624,Data Engineer,143392.48,147694.2544
785641,Data Engineer,154500.00,159135.0000
785648,Data Scientist,228531.25,235387.1875
785682,Data Scientist,162225.00,167091.7500


Technically you could write this with a lambda function:


In [7]:
df['salary_year_inflated'] = df.apply(lambda row: row['salary_year_avg'] *
                                      1.05 if 'Senior' in row['job_title_short'] else row['salary_year_avg'] * 1.03, axis=1)

df[pd.notna(df['salary_year_avg'])][['job_title_short',
                                     'salary_year_avg', 'salary_year_inflated']]

,job_title_short,salary_year_avg,salary_year_inflated
28,Data Scientist,112785.00,116168.5500
77,Data Engineer,144200.00,148526.0000
92,Data Engineer,123600.00,127308.0000
100,Data Scientist,235068.66,242120.7198
109,Data Analyst,91670.00,94420.1000
...,...,...,...
785624,Data Engineer,143392.48,147694.2544
785641,Data Engineer,154500.00,159135.0000
785648,Data Scientist,228531.25,235387.1875
785682,Data Scientist,162225.00,167091.7500


Example 3

Convert the `job_skills` from a generic object to an actual list object (_hint_ this is very important for later). Let's try doing that by just using `ast.literal_eval` and then look at our new column.

A reminder of what our `job_skills` column looks like now:


In [8]:
df['job_skills']

0                                                      None
1         ['r', 'python', 'sql', 'nosql', 'power bi', 't...
2         ['python', 'sql', 'c#', 'azure', 'airflow', 'd...
3         ['python', 'c++', 'java', 'matlab', 'aws', 'te...
4         ['bash', 'python', 'oracle', 'aws', 'ansible',...
                                ...                        
785736    ['bash', 'python', 'perl', 'linux', 'unix', 'k...
785737                       ['sas', 'sas', 'sql', 'excel']
785738                              ['powerpoint', 'excel']
785739    ['python', 'go', 'nosql', 'sql', 'mongo', 'she...
785740                                      ['aws', 'flow']
Name: job_skills, Length: 785741, dtype: object

In [9]:
df['job_skills'][1]

"['r', 'python', 'sql', 'nosql', 'power bi', 'tableau']"

Inspecting it this list is currently a string!


In [ ]:
type(df['job_skills'][1])  # It is still an object (string)

str

Let's look at the `literal_eval()` function from the Python Standard Library `ast` module.


In [11]:
import ast

ast.literal_eval(df['job_skills'][1])

['r', 'python', 'sql', 'nosql', 'power bi', 'tableau']

In [ ]:
type(ast.literal_eval(df['job_skills'][1]))  # It is a list now

list

**Debugging**

**This is an intentional mistake**

This is used to demonstrate debugging.

Error: This will return an error because in the `job_skills` column has NaN values.

Steps to Debug:

1. Look at the actual error, can you tell what the problem is?
2. If not, then look it up:
3. Use a chatbot like ChatGPT or Claude
4. Look it up using Google


In [ ]:
# Returns an error

# Convert string representation to actual list
# df['job_skills'] = df['job_skills'].apply(ast.literal_eval)

# df.head()

Since we have nan values, lets adjust our code to add in a condition to check if the value is not NaN.

- If it's not NaN it returns `True` and applies `ast.literal_eval()` function on it.
- if it's a Nan value then it returns `False` and the NaN value doesn't change.


In [ ]:
import ast

# Convert string representation to actual list, checking for NaN values first
df['job_skills'] = df['job_skills'].apply(
    lambda x: ast.literal_eval(x) if pd.notna(x) else x)

Now let's look at our `job_skills` column. You can see that it's actually a list.


In [15]:
df['job_skills']

0                                                      None
1                [r, python, sql, nosql, power bi, tableau]
2         [python, sql, c#, azure, airflow, dax, docker,...
3         [python, c++, java, matlab, aws, tensorflow, k...
4         [bash, python, oracle, aws, ansible, puppet, j...
                                ...                        
785736    [bash, python, perl, linux, unix, kubernetes, ...
785737                               [sas, sas, sql, excel]
785738                                  [powerpoint, excel]
785739    [python, go, nosql, sql, mongo, shell, mysql, ...
785740                                          [aws, flow]
Name: job_skills, Length: 785741, dtype: object

In [ ]:
type(df['job_skills'][1])  # It is a list now

list